In [1]:
from memory.unsafe import DTypePointer
from memory import memset_zero

In [30]:
struct Field[DT: DType, VW: Int]:
    var data: DTypePointer[DT]
    var gx: Int
    var gy: Int
    var gz: Int
    var dsize: Int
    
    fn __init__(inout self, gx: Int, gy: Int, gz: Int):
        self.dsize = (gx+2)*(gy+2)*(gz+2)
        self.data = DTypePointer[DT].alloc(self.dsize)
        memset_zero(self.data, self.dsize)
        self.gx = gx
        self.gy = gy
        self.gz = gz

    fn __copyinit__(inout self, other: Self):
        self.gx = other.gx
        self.gy = other.gy
        self.gz = other.gz
        self.data = DTypePointer[DT].alloc(other.dsize)
        self.dsize = other.dsize
        for i in range(self.dsize - self.dsize % VW,VW):
            self.data.simd_store[VW](i, other.data.simd_load[VW](i))
        for i in range(self.dsize - self.dsize % VW, self.dsize)
            self.data.simd_store[1](i, other.data.simd_load[1](i)):
        # needs to handle data inconmensurate with VW

    fn __add__(self, rhs: Field[DT,VW]) raises -> Field[DT,VW]:
        # test if the fields have the same sizes
        if self.gx != rhs.gx or self.gy != rhs.gy or self.gz != rhs.gz :
            raise ("cannot add fields with different grid sizes")
        let result = Field[DT,VW](self.gx, self.gy, self.gz)
        let p : Int
        let vwgx = self.gx - self.gx%VW
        for k in range(1,self.gz+1):
            for j in range(1,self.gy+1):
                print("in loop jk", j,k)
                for i in range (1,vwgx+1,VW):
                    p = k * (self.gx+2) * (self.gy+2) + j * (self.gx+2) + i
                    print (i,j,k,p, VW, vwgx, self.gx%VW)
                    print(self.data.simd_load[VW](p), rhs.data.simd_load[VW](p))
                    result.data.simd_store[VW](p, self.data.simd_load[VW](p) + rhs.data.simd_load[VW](p))
                    print(p, result.data.simd_load[VW](p), result.data.load(p))
                for i in range (vwgx+1,self.gx+1):
                    print("in loop 2", i, vwgx)
                    p = k * (self.gx+2) * (self.gy+2) + j * (self.gx+2) + i
                    result.data.simd_store[1](p, self.data.simd_load[1](p) + rhs.data.simd_load[1](p))
                    print("sum",p,result.data.load(p))
        return result

    fn __del__(owned self):
        self.data.free()
        print("done del")
        
    fn zero(inout self):
        memset_zero[DT](self.data, self.dsize)

    @always_inline
    fn __getitem__(self, x: Int, y: Int, z: Int) -> SIMD[DT,1]:
        let p = z * (self.gx+2) * (self.gy+2) + y * (self.gx+2) + x 
        print("get",p,self.data.load(p))
        return self.data.load(p)

    @always_inline
    fn load[nelts:Int=VW](self, x: Int, y: Int, z: Int) -> SIMD[DT,nelts]:
        return self.data.simd_load[nelts](z * (self.gx+2) * (self.gy+2) + y * (self.gx+2) + x)

    @always_inline
    fn __setitem__(self, x: Int, y: Int, z: Int, val: SIMD[DT,1]):
        return self.data.store(z * (self.gx+2) * (self.gy+2) + y * (self.gx+2) + x, val)

    @always_inline
    fn store[nelts:Int=VW](self, x: Int, y: Int, z: Int, val: SIMD[DT, nelts]):
        self.data.simd_store(z * (self.gx+2) * (self.gy+2) + y * (self.gx+2) + x, val)

error: Expression [30] wrapper:31:65: expected ':' after expression
        for i in range(self.dsize - self.dsize % VW, self.dsize)
                                                                ^

error: Expression [30]:25:67: statements must start at the beginning of a line
            self.data.simd_store[1](i, other.data.simd_load[1](i)):
                                                                  ^

expression failed to parse (no further compiler diagnostics)

In [34]:

let n=4
alias w=4
var a = Field[DType.float32,w](n,1,1)
var b = Field[DType.float32,w](n,1,1)
#var c = Field[DType.float32,w](n,1,1)

for i in range(1,n+1):
    a[i,1,1] = i
    b[i,1,1] = i*10
    print(i, a[i,1,1], b[i,1,1])

let c : Field[DType.float32,w] = a + b
#c.zero()
for i in range(1,n+1):
    print(i, c[i,1,1])



get 25 1.0
get 25 10.0
1 1.0 10.0
get 26 2.0
get 26 20.0
2 2.0 20.0
get 27 3.0
get 27 30.0
3 3.0 30.0
get 28 4.0
get 28 40.0
4 4.0 40.0
in loop 1 1
1 1 1 25 4 4 0
[1.0, 2.0, 3.0, 4.0] [10.0, 20.0, 30.0, 40.0]
25 [11.0, 22.0, 33.0, 44.0] 11.0
done del
get 25 11.0
1 11.0
get 26 22.0
2 22.0
get 27 33.0
3 33.0
get 28 44.0
4 44.0


In [16]:
var a: SIMD[DType.float64, 4]
var b: SIMD[DType.float64, 4]
var c: SIMD[DType.float64, 4]

a=1.0
b=3.33
c=a+b
print(a,b,c)

[1.0, 1.0, 1.0, 1.0] [3.3300000000000001, 3.3300000000000001, 3.3300000000000001, 3.3300000000000001] [4.3300000000000001, 4.3300000000000001, 4.3300000000000001, 4.3300000000000001]
